In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import requests

# Importing the dataset
weather_dataset = pd.read_csv('rearranged_balanced.csv')
print(weather_dataset)

# Shuffling the data
np.random.shuffle(weather_dataset.values)
print(weather_dataset.values)

# Conversion of the date field into integer format
day_dic = {
    'San': 1,
    'Man': 2,
    'Tues': 3,
    'Wedn': 4,
    'Thur': 5
}
print(day_dic)
print(weather_dataset['Day'])

# Formulating inputs
weather_inputs = np.column_stack((weather_dataset.Day.map(day_dic).values, weather_dataset.Time.values, weather_dataset.Temperature.values, weather_dataset.Humidity.values, weather_dataset.UV_level.values))
print(weather_inputs)

# Formulating outputs
weather_outputs = np.column_stack((weather_dataset.Rain.values, weather_dataset.Wind_Speed.values))
print(weather_outputs)

print()
print('Building the neural network')

# Preparing the neural network
weather_NN = keras.Sequential([
    keras.layers.Dense(128, input_shape=(5,), activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(2)  # Output layer with linear activation
])

# Compiling the neural network
weather_NN.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Fitting the model
weather_NN.fit(weather_inputs.astype(np.float32), weather_outputs.astype(np.float32), batch_size=4, epochs=10)

# Predicting the rain and wind speed
prediction = weather_NN.predict(np.column_stack((np.array([2]), np.array([160000]), np.array([29.5]), np.array([90.2]), np.array([0]))))

# Apply threshold for rain prediction
rain_threshold = 0.5
binary_rain_prediction = 1 if prediction[0][0] >= rain_threshold else 0

print('Prediction - Rain (Binary):', binary_rain_prediction)  # 0 or 1
print('Prediction - Wind Speed:', prediction[0][1])  # Predicted wind speed

# Send data to ThingSpeak
api_key = '0II1YQGDO1YRYQ87'
rain_field = binary_rain_prediction
wind_speed_field = prediction[0][1]
url = f'https://api.thingspeak.com/update?api_key={api_key}&field1={rain_field}&field2={wind_speed_field}'
response = requests.get(url)

if response.status_code == 200:
    print('Data sent to ThingSpeak successfully.')
else:
    print('Error sending data to ThingSpeak.')


        Day   Time  entry_id  Temperature  Humidity  UV_level  Rain  \
0       San  60000         1         31.2      87.1        22     1   
1       San  60020         2         31.2      87.3        23     1   
2       San  60040         3         31.2      87.3        32     1   
3       San  60100         4         31.2      87.2       322     1   
4       San  60120         5         31.2      87.0        33     1   
...     ...    ...       ...          ...       ...       ...   ...   
21596  Thur  55820     21597         30.8      86.5         0     0   
21597  Thur  55840     21598         30.8      86.5         0     0   
21598  Thur  55900     21599         30.8      86.5         0     0   
21599  Thur  55920     21600         30.8      86.6         0     0   
21600  Thur  55940     21601         30.9      87.0         0     0   

       Wind_Speed  
0            0.00  
1            0.00  
2            0.00  
3            0.00  
4            2.85  
...           ...  
21596  